Copyright 2021 D Bazzett  
The purpose of this code is to convert our flow forecasts for 10-year and worst case + 10%, 20%, and 30% into friendly netcdf files that can be read by Yan's code.

In [1]:
# !pip install pandas==0.25

In [2]:
import pandas as pd
print(pd.__version__)
import xarray as xr


0.25.0


C:\Users\David\Anaconda3\lib\site-packages\xarray\core\merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [23]:
# file = './ModelFlow/MODEL_FLOW.csv'
file = './ModelFlow/res_linear.csv'
flow = pd.read_csv(file,encoding = 'unicode_escape')
flow

,COMID,R²,Slope,POINT_X,POINT_Y,dis_a00,dis_a10,dis_a20,dis_a30,dis_a40,dis_a50,dis_w00,dis_w10,dis_w20,dis_w30,dis_w40,dis_w50
0,6202418,NaN,0.000000,-74.654699,41.334533,0.654444,0.719889,0.785333,0.850778,0.916222,0.981667,2.850000,3.135000,3.420000,3.705000,3.990000,4.275000
1,6202516,NaN,0.000000,-74.631395,41.318358,0.420411,0.462452,0.504493,0.546534,0.588575,0.630616,1.300000,1.430000,1.560000,1.690000,1.820000,1.950000
2,6202540,NaN,0.000000,-74.613370,41.318597,1.178219,1.296041,1.413863,1.531685,1.649507,1.767329,4.520000,4.972000,5.424000,5.876000,6.328000,6.780000
3,6202556,NaN,0.000000,-74.643862,41.318933,1.351309,1.486440,1.621571,1.756702,1.891833,2.026964,7.150000,7.865000,8.580000,9.295000,10.010000,10.725000
4,6202558,NaN,0.000000,-74.651496,41.316335,1.667742,1.834516,2.001290,2.168064,2.334839,2.501613,6.550000,7.205000,7.860000,8.515000,9.170000,9.825000
5,6202568,NaN,0.000000,-74.660144,41.314816,1.036600,1.140260,1.243920,1.347580,1.451240,1.554900,4.840000,5.324000,5.808000,6.292000,6.776000,7.260000
6,6202572,NaN,0.000000,-74.607113,41.307307,2.033043,2.236348,2.439652,2.642956,2.846261,3.049565,9.890000,10.879000,11.868000,12.857000,13.846000,14.835000
7,6202616,NaN,0.000000,-74.625339,41.308549,1.643778,1.808156,1.972533,2.136911,2.301289,2.465667,6.170000,6.787000,7.404000,8.021000,8.638000,9.255000
8,6202654,NaN,0.000000,-74.651314,41.305938,3.755432,4.130975,4.506518,4.882062,5.257605,5.633148,18.130000,19.943000,21.756000,23.568999,25.381999,27.194999
9,6202662,NaN,0.000000,-74.661232,41.303091,1.104035,1.214439,1.324842,1.435246,1.545649,1.656053,3.810000,4.191000,4.572000,4.953000,5.334000,5.715000


In [20]:
len(flow.index)

11311

In [29]:
from datetime import datetime

names = ['dis_a00', 'dis_a10', 'dis_a20', 'dis_a30', 'dis_a40', 'dis_a50',\
         'dis_w00', 'dis_w10', 'dis_w20', 'dis_w30', 'dis_w40', 'dis_w50',]

x=0
y=1
for name in names:
    set_time = datetime(2020, 1, 1, 1, 0, 0)
    print(name,end='')
    subset = flow[['COMID',name]]
    subset = subset.rename(columns={'COMID': 'feature_id', name: 'streamflow'})
    
    flowx = xr.Dataset(
            data_vars=dict(
                streamflow = (['feature_id'], subset.streamflow.values)
            ),
            coords=dict(
                time=(['time'],[set_time]),
                reference_time=(['reference_time'],[set_time]),
                feature_id=subset.feature_id.values,
            ),
            attrs=dict(
            model_output_valid_time = '2020-01-0' + str(y) + '_0' + str(x) + ':00:00',
            model_initialization_time = '2020-01-01_00:00:00',
            )
    ) 
    print(' 2020-01-0' + str(y) + '_0' + str(x) + ':00:00')
    flowx.to_netcdf('./ModelFlow/' + name + '.nc')
    x += 1
    if x == 6:
        y = 2
        x = 0

dis_a00 2020-01-01_00:00:00
dis_a10 2020-01-01_01:00:00
dis_a20 2020-01-01_02:00:00
dis_a30 2020-01-01_03:00:00
dis_a40 2020-01-01_04:00:00
dis_a50 2020-01-01_05:00:00
dis_w00 2020-01-02_00:00:00
dis_w10 2020-01-02_01:00:00
dis_w20 2020-01-02_02:00:00
dis_w30 2020-01-02_03:00:00
dis_w40 2020-01-02_04:00:00
dis_w50 2020-01-02_05:00:00


In [7]:
# Dimensions     feature_id: 2729077 reference_time: 1 time: 1
flowx

<xarray.Dataset>
Dimensions:         (feature_id: 11311, reference_time: 1, time: 1)
Coordinates:
  * time            (time) datetime64[ns] 2020-01-01T01:00:00
  * reference_time  (reference_time) datetime64[ns] 2020-01-01T01:00:00
  * feature_id      (feature_id) int64 6202418 6202516 ... 932040265 932040275
Data variables:
    streamflow      (feature_id) float64 4.275 1.95 6.78 ... 4.905 6.06 4.89
Attributes:
    model_output_valid_time:    2020-01-01_012:00:00
    model_initialization_time:  2020-01-01_00:00:00

In [8]:
for var in flowx.variables.values():
    print(var)

<xarray.Variable (feature_id: 11311)>
array([4.275, 1.95 , 6.78 , ..., 4.905, 6.06 , 4.89 ])
<xarray.IndexVariable 'time' (time: 1)>
array(['2020-01-01T01:00:00.000000000'], dtype='datetime64[ns]')
<xarray.IndexVariable 'reference_time' (reference_time: 1)>
array(['2020-01-01T01:00:00.000000000'], dtype='datetime64[ns]')
<xarray.IndexVariable 'feature_id' (feature_id: 11311)>
array([  6202418,   6202516,   6202540, ..., 932040263, 932040265, 932040275],
      dtype=int64)
